REQUIRMENTS

In [ ]:
%pip install tflearn

In [ ]:
%pip uninstall tensorflow

In [ ]:
%pip install tensorflow

In [ ]:
%pip install nltk

In [ ]:
#nltk.download()

RULE BASED MODEL

In [1]:
# Importing modules
import re
from nltk.corpus import wordnet

# Building a list of Keywords
list_words=[
    'hello','bye',
    'suicide',
    'angry','anxious','miserable','ashamed','depressed','defeated','frustrated'
    'good','happy',
    ]
list_syn={}
for word in list_words:
    synonyms=[]
    for syn in wordnet.synsets(word):
        for lem in syn.lemmas():
            # Remove any special characters from synonym strings
            lem_name = re.sub('[^a-zA-Z0-9 \n\.]', ' ', lem.name())
            synonyms.append(lem_name)
    list_syn[word]=set(synonyms)
print (list_syn)

# Building dictionary of Intents & Keywords
keywords={}
keywords_dict={}
# Defining a new key in the keywords dictionary
keywords['greetings']=[]
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(list_syn['hello']):
    keywords['greetings'].append('.*\\b'+synonym+'\\b.*')

# Defining a new key in the keywords dictionary
keywords['suicide']=[]
# Populating the values in the keywords dictionary with synonyms of keywords formatted with RegEx metacharacters 
for synonym in list(list_syn['suicide']):
    keywords['suicide'].append('.*\\b'+synonym+'\\b.*')
for intent, keys in keywords.items():
    # Joining the values in the keywords dictionary with the OR (|) operator updating them in keywords_dict dictionary
    keywords_dict[intent]=re.compile('|'.join(keys))
print (keywords_dict)

# Building a dictionary of responses
responses={
    'greetings':'Hello! So, tell me how have you been feeling?',
    'suicide':'If you are feeling vulnerable and need help, call 112 or 914 590 055 to seek help.',
    'fallback':'I dont quite understand. Could you repeat that?',
}

def get_response(msg):
    matched_intent = None 
    for intent,pattern in keywords_dict.items():
        # Using the regular expression search function to look for keywords in user input
        if re.search(pattern, msg): 
            # if a keyword matches, select the corresponding intent from the keywords_dict dictionary
            matched_intent=intent  
    # The fallback intent is selected by default
    key='fallback' 
    if matched_intent in responses:
        # If a keyword matches, the fallback intent is replaced by the matched intent as the key for the responses dictionary
        key = matched_intent
    # The chatbot prints the response that matches the selected intent
    print (responses[key]) 
    return responses[key]



{'hello': {'hello', 'howdy', 'hullo', 'hi', 'how do you do'}, 'bye': {'good bye', 'goodbye', 'sayonara', 'cheerio', 'good by', 'bye bye', 'auf wiedersehen', 'arrivederci', 'so long', 'adios', 'goodby', 'adieu', 'bye', 'good day', 'au revoir', 'pass'}, 'suicide': {'felo de se', 'suicide', 'self destruction', 'self annihilation'}, 'angry': {'angry', 'furious', 'raging', 'wild', 'tempestuous'}, 'anxious': {'anxious', 'uneasy', 'unquiet', 'nervous', 'queasy', 'dying'}, 'miserable': {'low down', 'abject', 'poor', 'execrable', 'hapless', 'piteous', 'miserable', 'scurvy', 'pitiable', 'scummy', 'measly', 'paltry', 'suffering', 'deplorable', 'pitiful', 'low', 'wretched', 'misfortunate', 'woeful', 'pathetic'}, 'ashamed': {'ashamed'}, 'depressed': {'low spirited', 'down in the mouth', 'get down', 'dispirited', 'lower', 'gloomy', 'depress', 'down', 'dispirit', 'depressed', 'downhearted', 'downcast', 'press down', 'dismay', 'deject', 'grim', 'demoralise', 'cast down', 'low', 'demoralize', 'blue'}, 

CHAT GUI

In [2]:
from tkinter import *

BG_GRAY = "#ABB2B9"
BG_COLOR = "#17202A"
TEXT_COLOR = "#EAECEE"

FONT = "Helvetica 14"
FONT_BOLD = "Helvetica 13 bold"

bot_name = "Kai"

class ChatGUI:
    
    def __init__(self):
        self.window = Tk()
        self._setup_main_window()
        self._insert_initial_message("Hello!\
            \nMy name is Kai and I'm going to give you support to your self-directed CBT Therapy session.\
            \nSo, how have you been feeling?")
        
    def run(self):
        self.window.mainloop()
        
    def _setup_main_window(self):
        self.window.title("Chat")
        self.window.resizable(width=False, height=False)
        self.window.configure(width=470, height=550, bg=BG_COLOR)
        
        # head label
        head_label = Label(self.window, bg=BG_COLOR, fg=TEXT_COLOR,
                           text="Kai", font=FONT_BOLD, pady=10)
        head_label.place(relwidth=1)
        
        # tiny divider
        line = Label(self.window, width=450, bg=BG_GRAY)
        line.place(relwidth=1, rely=0.07, relheight=0.012)
        
        # text widget
        self.text_widget = Text(self.window, width=20, height=2, bg=BG_COLOR, fg=TEXT_COLOR,
                                font=FONT, padx=5, pady=5)
        self.text_widget.place(relheight=0.745, relwidth=1, rely=0.08)
        self.text_widget.configure(cursor="arrow", state=DISABLED)
        
        # scroll bar
        scrollbar = Scrollbar(self.text_widget)
        scrollbar.place(relheight=1, relx=0.974)
        scrollbar.configure(command=self.text_widget.yview)
        
        # bottom label
        bottom_label = Label(self.window, bg=BG_GRAY, height=80)
        bottom_label.place(relwidth=1, rely=0.825)
        
        # message entry box
        self.msg_entry = Entry(bottom_label, bg="#2C3E50", fg=TEXT_COLOR, font=FONT)
        self.msg_entry.place(relwidth=0.74, relheight=0.06, rely=0.008, relx=0.011)
        self.msg_entry.focus()
        self.msg_entry.bind("<Return>", self._on_enter_pressed)
        
        # send button
        send_button = Button(bottom_label, text="Send", font=FONT_BOLD, width=20, bg=BG_GRAY,
                             command=lambda: self._on_enter_pressed(None))
        send_button.place(relx=0.77, rely=0.008, relheight=0.06, relwidth=0.22)
    
    def _on_enter_pressed(self, event):
        msg = self.msg_entry.get()
        self._insert_message(msg, "You")
       
    def _insert_message(self, msg, sender):
        if not msg:
            return
        
        self.msg_entry.delete(0, END)
        msg1 = f"{sender}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
        
        msg2 = f"{bot_name}: {get_response(msg)}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg2)
        self.text_widget.configure(state=DISABLED)
        
        self.text_widget.see(END)
          
    def _insert_initial_message(self, msg):
        self.msg_entry.delete(0, END)
        msg1 = f"{bot_name}: {msg}\n\n"
        self.text_widget.configure(state=NORMAL)
        self.text_widget.insert(END, msg1)
        self.text_widget.configure(state=DISABLED)
     
if __name__ == "__main__":
    app = ChatGUI()
    app.run()

Hello! So, tell me how have you been feeling?
I dont quite understand. Could you repeat that?
